In [1]:
import ROOT as rt
import numpy as np

Welcome to JupyROOT 6.10/08


In [2]:
histo_file_loc = '../data/_root/histos_1Dq2.root'

In [3]:
tf = rt.TFile(histo_file_loc, 'READ')

In [4]:
expected = {}
expected['mu'] = 0.99
expected['tau'] = 0.01

In [5]:
if np.sum(expected.values()) != 1:
    raise

In [6]:
name = {}
n_keys = 0
for k in tf.GetListOfKeys():
    if 'data' in k.GetName():
        name['data'] = k.GetName()
    else:
        for ke in expected.keys():
            if ke in k.GetName():
                name[ke] = k.GetName()
    
    n_keys += 1

In [7]:
histo = {}
for k,n in name.iteritems():
    histo[k] = tf.Get(n)

In [8]:
combine_histo_file_loc =  histo_file_loc.replace('.root', '_forCombine.root')
tfc = rt.TFile(combine_histo_file_loc, 'RECREATE')
for k, h in histo.iteritems():
    print k
    new_name = k
    if k == 'data':
        new_name += '_obs'
    h_aux = h.Clone(new_name)
    if k != 'data':
        h_aux.Scale(histo['data'].Integral()*expected[k]/h.Integral())
        
    h_aux.Write()
tfc.Close()

mu
tau
data


# Write the card

In [9]:
fc = open('../CombineCards/q2_MuTau.txt', 'w')

In [10]:
card = 'imax 1\n'
card += 'jmax {}\n'.format(len(expected.keys())-1)
card += 'kmax 0\n'
card += '---------------\n'
card += 'shapes * * ' + combine_histo_file_loc + ' $PROCESS\n'
card += '---------------\n'
card += 'bin bin1\n'
card += 'observation {:.0f}\n'.format(histo['data'].Integral())
card += '---------------\n'
card += 'bin ' + ' '.join(['bin1']*len(expected.keys())) + '\n'
card += 'process ' + ' '.join(expected.keys()) + '\n'
card += 'process ' + ' '.join([str(i) for i in range(len(expected.keys()))]) + '\n'
card += 'rate ' + ' '.join([str(histo['data'].Integral()*expected[k]) for k in expected.keys()]) + '\n'

In [11]:
print card

imax 1
jmax 1
kmax 0
---------------
shapes * * ../data/_root/histos_1Dq2_forCombine.root $PROCESS
---------------
bin bin1
observation 229542
---------------
bin bin1 bin1
process mu tau
process 0 1
rate 227246.58 2295.42



In [12]:
fc.write(card)
fc.close()